## distributed demo
this notebook shows how to run distributed julia programs.   it assumes you have a version of julia running on a machine on your local network.   In this case the worker container was run with the command 

<pre>
docker run -it -d -p 3456:22 dbgannon/juliacloud2
</pre>
the IP address of the machine where this was run is known to be 10.0.0.29.

In [ ]:
using Distributed
workers()

In [ ]:
using PyCall

In [ ]:
workervec = [("jovyan@10.0.0.29:3456", 2)]

In [ ]:
addprocs(workervec ; sshflags=`-i /home/jovyan/.ssh/pubkey`, tunnel=true, exename=`/usr/local/bin/julia`)

In [ ]:
@everywhere using Distributed

In [ ]:
@everywhere using PyCall

In [ ]:
@everywhere using Sockets

In [ ]:
getipaddr()

In [ ]:
@everywhere function putInTable(row, data)
    println("hi from node ",myid() )
    println("my ip address is ", getipaddr())
    println("the row is ", row, " and the data is ", data)
end

In [ ]:
workers()

In [ ]:
x = [@spawnat i putInTable( i*10 ,"foo foo") for i in 4:5]

In [ ]:
fetch(x)

In [ ]:
ch = RemoteChannel(()->Channel{String}(1000), 1)

In [ ]:
@everywhere function listen(ch)
    println("listen started")
    row = 0
    id = "1"
    while id != "-1"
        id = take!(ch)
        x = string("out from ",string(myid())," val=", string(id), " row =", string(row))
        println(x)
        #putInTable(tbl,row, x )
        row = row+1
    end
    println("listen done")
end

In [ ]:
x = [@spawnat i listen(ch) for i in workers()]

In [ ]:
function putInCh(ch)
    for i=1:4
        put!(ch, string(i))
        println(i)
    end
end

In [ ]:
putInCh(ch)

In [ ]:
put!(ch, string(-1))

In [ ]:
workers()